In [7]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableLambda, RunnableParallel


load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [2]:
summary_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie critic."),
        ("human", "Provide a brief summary of the movie {movie_name}."),
    ]
)


In [3]:
# Define plot analysis step
def analyze_plot(plot):
    plot_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a movie critic."),
            ("human", "Analyze the plot: {plot}. What are its strengths and weaknesses?"),
        ]
    )
    return plot_template.format_prompt(plot=plot)

# Define character analysis step
def analyze_characters(characters):
    character_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a movie critic."),
            ("human", "Analyze the characters: {characters}. What are their strengths and weaknesses?"),
        ]
    )
    return character_template.format_prompt(characters=characters)


In [4]:

# Combine analyses into a final verdict
def combine_verdicts(plot_analysis, character_analysis):
    return f"Plot Analysis:\n{plot_analysis}\n\nCharacter Analysis:\n{character_analysis}"

In [6]:
# Simplify branches with LCEL
plot_branch_chain = (
    RunnableLambda(lambda x: analyze_plot(x)) | llm | StrOutputParser()
)

character_branch_chain = (
    RunnableLambda(lambda x: analyze_characters(x)) | llm | StrOutputParser()
)

In [8]:
chain = (
    summary_template
    | llm
    | StrOutputParser()
    | RunnableParallel(branches={"plot": plot_branch_chain, "characters": character_branch_chain})
    | RunnableLambda(lambda x: combine_verdicts(x["branches"]["plot"], x["branches"]["characters"]))
)


In [9]:
result = chain.invoke({"movie_name": "Inception"})

print(result)

Plot Analysis:
Alright, let's break down the plot of *Inception* from a critical perspective.

**Strengths:**

*   **High-Concept Premise:** The idea of dream-sharing and inception is inherently fascinating and original. It immediately grabs the audience's attention and promises a unique cinematic experience. The concept itself is a major strength, allowing for exploration of complex psychological themes.
*   **Intricate World-Building:** Nolan meticulously constructs the rules and mechanics of the dream world. The layers of dreams, the concept of projections, the effects of gravity, and the use of totems to distinguish reality from dreams – all these elements are carefully thought out and visually compelling. This detailed world-building creates a sense of immersion and allows the narrative to become increasingly complex.
*   **Heist Thriller Structure:** Framing the story as a heist provides a familiar and engaging structure. The team assembles, plans the operation, faces challenges,